In [1]:
import os 

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

e:\langgraph_end_to_end\genai_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.0-pro')

In [7]:
llm.invoke("Hi,I am shoaib!1")

AIMessage(content='Hi Shoaib! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-90ed1b98-d99e-4980-ace1-0ff0193d6110-0', usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}})

### Wikipedia search tool

In [8]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [9]:
api_wrapper = WikipediaAPIWrapper(top_k_results = 5, doc_content_char_max = 100)

In [10]:
tool = WikipediaQueryRun(api_wrapper = api_wrapper)

In [11]:
tool.name

'wikipedia'

In [12]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [13]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [14]:
print(tool.run({"query":"What is langchain"}))

Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.



Page: Prompt injection
Summary: Prompt injection is a family of related computer security exploits carried out by getting a machine learning model (such as an LLM) which was trained to follow human-given instructions to follow instructions provided by a malicious user. This stands in contrast to the intended operation of instruction-following systems, wherein t

## Create Custom tool

In [15]:
from langchain.agents import tool
@tool
def get_word_length(word:str)->int:
    '''Returns the length of a word'''
    return len(word)

In [16]:
get_word_length.invoke("Shoaib")

6

In [17]:
@tool 
def multiply(a:int,b:int)->int:
    '''Multiply two numbers'''
    return a*b

In [18]:
multiply.invoke({"a":10,"b":20})

200

In [19]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Concept of AGENTS


In [20]:
# This AGENT class from the first verion of langchain
from langchain.agents import AgentType, load_tools, initialize_agent

In [21]:
tool = load_tools(["wikipedia"],llm = llm)

In [22]:
agent = initialize_agent(tool,llm,agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose = True,handle_parsing_errors=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_14300\1277187307.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(tool,llm,agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose = True,handle_parsing_errors=True)


In [23]:
agent.run("What is langchain and who created llm model")

C:\Users\Asus\AppData\Local\Temp\ipykernel_14300\3296016223.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is langchain and who created llm model")




> Entering new AgentExecutor chain...
I should look up what langchain is.
Action: wikipedia
Action Input: What is langchain
Observation: Page: Retrieval-augmented generation
Summary: Retrieval Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.



Page: Prompt injection
Summary: Prompt injection is a family of related computer security exploits carried out by getting a machine learning model (such as an LLM) which was trained to follow human-given instructions to foll

'I apologize, but the provided context does not contain any information about who created the LLM model.'

In [24]:
agent.run("What is the gdp of india country?")



> Entering new AgentExecutor chain...
Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Final Answer: I cannot answer this question, please try again with a valid command.

> Finished chain.


'I cannot answer this question, please try again with a valid command.'

In [25]:
# Second Tool 
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro')
from langchain_community.tools.tavily_search import TavilySearchResults

In [26]:
search = TavilySearchResults()

In [27]:
search.invoke("What is the weather in Lucknow?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Lucknow', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 26.85, 'lon': 80.917, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734021427, 'localtime': '2024-12-12 22:07'}, 'current': {'last_updated_epoch': 1734021000, 'last_updated': '2024-12-12 22:00', 'temp_c': 13.9, 'temp_f': 57.1, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 292, 'wind_dir': 'WNW', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 20, 'cloud': 0, 'feelslike_c': 13.2, 'feelslike_f': 55.8, 'windchill_c': 13.2, 'windchill_f': 55.8, 'heatindex_c': 13.9, 'heatindex_f': 57.1, 'dewpoint_c': -8.5, 'dewpoint_f': 16.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 14.1, 'gust_kph': 22.7}}"},
 {'url': 'https://www.livemint.com/news/india/lucknow-weather-forecast-today-on-

In [28]:
tools = [search]

In [29]:
from langchain import hub

In [30]:
prompt = hub.pull("hwchase17/openai-functions-agent")

e:\langgraph_end_to_end\genai_env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [31]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm,tools,prompt)

In [33]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [34]:
agent_executor.invoke({"input":"Hello how are you?"})



> Entering new AgentExecutor chain...
I am doing well, how are you?


> Finished chain.


{'input': 'Hello how are you?', 'output': 'I am doing well, how are you?\n'}

In [35]:
agent_executor.invoke({"input":"What is the weather in Lucknow?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in Lucknow'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Lucknow', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 26.85, 'lon': 80.917, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734024109, 'localtime': '2024-12-12 22:51'}, 'current': {'last_updated_epoch': 1734023700, 'last_updated': '2024-12-12 22:45', 'temp_c': 13.9, 'temp_f': 57.1, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.7, 'wind_kph': 10.8, 'wind_degree': 292, 'wind_dir': 'WNW', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 20, 'cloud': 0, 'feelslike_c': 13.2, 'feelslike_f': 55.8, 'windchill_c': 13.2, 'windchill_f': 55.8, 'heatindex_c': 13.9, 'heatindex_f': 57.1, 'dewpoint_c': -8.5, 'dewpoint_f': 16.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, '

{'input': 'What is the weather in Lucknow?',
 'output': 'The weather in Lucknow, India is currently clear with a temperature of 13.9°C.  The wind is blowing from the WNW at 10.8 kph. The humidity is 20%.\n'}

### Agent Type : Function calling
A tool calling agent refers to an AI system or framewrok that dynamically invokes external tools or API to accomplish task as part of its decision making process.Instead of relying solely on internal reasoning or knowledge, the agent calls specific tools to perform actions like calculations, searches, or other operations. This type of agent is often used in real-world applications where tasks require interaction with external system.

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [37]:
search = TavilySearchResults()

In [39]:
search.invoke("What is the weather in Mumbai?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734025512, 'localtime': '2024-12-12 23:15'}, 'current': {'last_updated_epoch': 1734025500, 'last_updated': '2024-12-12 23:15', 'temp_c': 27.3, 'temp_f': 81.1, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 10.3, 'wind_kph': 16.6, 'wind_degree': 16, 'wind_dir': 'NNE', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 45, 'cloud': 0, 'feelslike_c': 28.8, 'feelslike_f': 83.9, 'windchill_c': 25.2, 'windchill_f': 77.4, 'heatindex_c': 26.5, 'heatindex_f': 79.6, 'dewpoint_c': 17.2, 'dewpoint_f': 62.9, 'vis_km': 2.5, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 15.8, 'gust_kph': 25.4}}"},
 {'url': 'https://www.hindustantimes.com/cities/mumbai-news/mumbai-weather-today

In [40]:
from langchain import hub

In [41]:
prompt = hub.pull("hwchase17/openai-functions-agent")

e:\langgraph_end_to_end\genai_env\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [42]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [43]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [45]:
# In simple tool calling if user is giving simple message then also it is calling the tool.
# like agent_executor.invoke({"input":"Hello how are you?"})

## Adding One more agent

In [84]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.tools.tavily_search import TavilySearchResults

In [85]:
search = TavilySearchResults()

In [86]:
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs = loader.load()

In [87]:
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="\n\n\nLangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024\n\n\n\n\n\nLLMs have paved the way for the dev

In [88]:
documents = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200).split_documents(docs)

In [89]:
documents

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content='LangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024'),
 Document(metadata={'source': 'https://blog.langc

In [90]:
vector = FAISS.from_documents(documents,embedding)

In [91]:
retriever = vector.as_retriever()

In [92]:
retriever.invoke("What is langgraph studio?")

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="is needed to make the development process easier. Building LLM applications differs from traditional software development, requiring different tooling outside of the traditional code editor.Coding is still important to developing LLM applications — after all, production-ready LangGraph applications have complicated custom logic in the nodes and edges of the graphs that are created. We don't aim to replace code editors but, instead, to augment the development experience with tools tailored for LangGraph applications. LangGraph Studio facilitates this by making it easy to visualize and interact with agent graphs, even if deve

In [93]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_studio",
    "Search for information about langgraph.For any questions about langgraph"
)

In [112]:
# tools = [search,retriever_tool]
from langchain.tools import Tool
tools = [
    Tool(
        name="search",
        func=search,
        description="Use this tool for general web search queries, like finding the weather or news."
    ),
    Tool(
        name="retriever_tool",
        func=retriever_tool,
        description="Search for information about langgraph.For any questions about langgraph"
    ),
]


In [113]:
tools

[Tool(name='search', description='Use this tool for general web search queries, like finding the weather or news.', func=TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))),
 Tool(name='retriever_tool', description='Search for information about langgraph.For any questions about langgraph', func=Tool(name='langgraph_studio', description='Search for information about langgraph.For any questions about langgraph', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000220FEB59E10>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000220BCD7CBE0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_r

In [114]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm,tools,prompt)

In [115]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [116]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(agent = agent,tools=tools, verbose = True)

In [117]:
agent_executor.invoke({"input":"What is weather in Mumbai?"})



> Entering new AgentExecutor chain...

Invoking: `search` with `weather in Mumbai`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734026826, 'localtime': '2024-12-12 23:37'}, 'current': {'last_updated_epoch': 1734026400, 'last_updated': '2024-12-12 23:30', 'temp_c': 27.2, 'temp_f': 81.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 10.3, 'wind_kph': 16.6, 'wind_degree': 16, 'wind_dir': 'NNE', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 45, 'cloud': 0, 'feelslike_c': 28.7, 'feelslike_f': 83.7, 'windchill_c': 25.2, 'windchill_f': 77.4, 'heatindex_c': 26.5, 'heatindex_f': 79.6, 'dewpoint_c': 17.2, 'dewpoint_f': 62.9, 'vis_km': 2.5, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 15.8, 'gust_kph': 25.4}

{'input': 'What is weather in Mumbai?',
 'output': 'The weather in Mumbai is currently overcast with a temperature of 27.2°C (81°F). The wind is blowing from the north-northeast at 16.6 kph.  The humidity is 45%.\n'}

In [118]:
agent_executor.invoke({"input":"What is a langgraph studio and how to use it?"})



> Entering new AgentExecutor chain...
LangGraph studio is Langchain's visual tool that allows you to easily create, modify, visualize, and test LLM workflows called LangGraphs.  You can drag and drop different components, connect them together and build complex logic without writing code.  It also helps with debugging and understanding the flow of your application.  More information can be found in the Langchain documentation.


> Finished chain.


{'input': 'What is a langgraph studio and how to use it?',
 'output': "LangGraph studio is Langchain's visual tool that allows you to easily create, modify, visualize, and test LLM workflows called LangGraphs.  You can drag and drop different components, connect them together and build complex logic without writing code.  It also helps with debugging and understanding the flow of your application.  More information can be found in the Langchain documentation.\n"}

In [119]:
agent_executor.invoke({"input":"What is the captain of Indian Cricket team?"})



> Entering new AgentExecutor chain...

Invoking: `search` with `captain of Indian Cricket team`


[{'url': 'https://en.wikipedia.org/wiki/List_of_India_national_cricket_captains', 'content': "Mithali Raj captained India in the first WT20I on 5 August 2006 against England at Derby.[16]\nKrishnamachari Srikkanth was the captain for the first youth test for India U-19 team against Pakistan U-19 which began on 20 January 1979 at Wankhede Stadium in Bombay while Ravi Shastri captained India U-19 in the first youth ODI in 1981 against England U-19.[17][18]\nHarmanpreet Kaur is current captain of the women's team and Rohit Sharma is current captain of men's team in all formats.\n During the match the captain decides the team's batting order, who will bowl each over, and where each fielder will be positioned.[1][2]\nCricket was introduced to the Indian subcontinent by the English in the 17th and 18th centuries.[3] Board of Control for Cricket in India (BCCI) is the national governing body of

{'input': 'What is the captain of Indian Cricket team?',
 'output': "Harmanpreet Kaur is the current captain of the India women's cricket team, and Rohit Sharma is the current captain of the India men's cricket team.\n"}

In [82]:
search.invoke("What is weather in Mumbai?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734025512, 'localtime': '2024-12-12 23:15'}, 'current': {'last_updated_epoch': 1734025500, 'last_updated': '2024-12-12 23:15', 'temp_c': 27.1, 'temp_f': 80.8, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 10.3, 'wind_kph': 16.6, 'wind_degree': 16, 'wind_dir': 'NNE', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 45, 'cloud': 0, 'feelslike_c': 28.6, 'feelslike_f': 83.4, 'windchill_c': 25.2, 'windchill_f': 77.4, 'heatindex_c': 26.5, 'heatindex_f': 79.6, 'dewpoint_c': 17.2, 'dewpoint_f': 62.9, 'vis_km': 2.5, 'vis_miles': 1.0, 'uv': 0.0, 'gust_mph': 15.8, 'gust_kph': 25.4}}"},
 {'url': 'https://www.hindustantimes.com/cities/mumbai-news/mumbai-weather-today